# Toys
The goal is to experiment the capabilities of RL. Document each usefull functionalitiy.
This code aim to try:
- different environement
- different agent
- different hardware
- Monitoring learning
- evaluating and comparing
- Complete a table with the KPIs

## Dependecies
- [Gymansium](https://gymnasium.farama.org/): Environments
- stable-baseline3: Agents 

In [9]:
!pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 11.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.0
    Uninstalling numpy-1.23.0:
      Successfully uninstalled numpy-1.23.0


In [ ]:
!pip install stable-baselines3
!pip install panda-gym
!pip install gymnasium[box2d]

## Device

In [2]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cpu


## Setup

### Setup agent

In [3]:
policy = 'MlpPolicy' # Configure
# 'MlpPolicy'
# 'CnnPolicy'
# 'MultiInputPolicy'

verbose = 0

name = 'ppo_lunar_v2' # Configure

path = f"./{name}/model/{name}"

tensorboard_log = f"./{name}/t_logs/"
tb_log_name = 'runs'

log_dir = f"./{name}/logs/"

### Setup environment

In [4]:
env_id = 'LunarLander-v2' # Configure
# 'LunarLander-v2'
# 'CartPole-v1'
# 'PandaReachDense-v3'

max_episode_steps = 1000

render_mode = 'rgb_array'
# 'human'
# 'rgb_array'

def make_env():
    e = gym.make(env_id, max_episode_steps=max_episode_steps, render_mode=render_mode)
    e = Monitor(e)
    return e

num_envs = 4

env_path = f'{name}/env/vec_normalize.pkl'

## Environment

### Make single environment

In [15]:
env = gym.make(env_id,max_episode_steps, render_mode=render_mode)

### Create vectorize environment

In [ ]:

env = DummyVecEnv([make_env for _ in range(num_envs)])

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.0)

env_dir = os.path.dirname(env_path)
if not os.path.exists(env_dir):
    os.makedirs(env_dir)  # 
env.save(env_path)

AttributeError: 'DummyVecEnv' object has no attribute 'save'

### Load vectorize environment

In [6]:
env = VecNormalize.load(env_path, DummyVecEnv([make_env for _ in range(num_envs)]))


pybullet build time: Nov 28 2023 23:45:17


argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


## Monitoring

In [11]:
class TensorboardCallback(BaseCallback):
    
    def __init__(self, verbose=0):
        super().__init__(verbose)

    def _on_step(self):
        exploration_rate = self.model.exploration_rate
        learning_rate = self.model.learning_rate
        self.logger.record("exploration_rate", exploration_rate)
        self.logger.record("learning_rate", learning_rate)
        return True
    
class HParamCallback(BaseCallback):

    def _on_training_start(self):
        hparam_dict = {
            "algorithm":self.model.__class__.__name__,
            "learning rate": self.model.learning_rate,
            "gamma": self.model.gamma
        }
        metric_dict = {
            "rollout/ep_len_mean":0,
            "train/value_loss": 0.0
        }
        self.logger.record(
            "hparams",
            HParam(hparam_dict, metric_dict),
            exclude=("stdout","log","json","csv")
        )

    def _on_step(self):
        return True
    
class SaveOnBestTrainingRewardCallback(BaseCallback):
    
    def __init__(self, check_freq:int, log_dir:str, path:str,verbose = 1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = path
        self.best_mean_reward = -np.inf

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:

          # Retrieve training reward
          x, y = ts2xy(load_results(self.log_dir), "timesteps")
          if len(x) > 0:
              # Mean training reward over the last 100 episodes
              mean_reward = np.mean(y[-100:])
              if self.verbose >= 1:
                print(f"Num timesteps: {self.num_timesteps}")
                print(f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}")

              # New best model, you could save the agent here
              if mean_reward > self.best_mean_reward:
                  self.best_mean_reward = mean_reward
                  # Example for saving best model
                  if self.verbose >= 1:
                    print(f"Saving new best model to {self.save_path}")
                  self.model.save(self.save_path)

        return True
        
class SaveOnStep(BaseCallback):
    def __init__(self, steps: int, path: str, verbose: int = 0):
        super().__init__(verbose)
        self.steps = steps
        self.save_path = path

    def _on_step(self) -> bool:
        # Check if the current step matches the saving frequency
        if self.n_calls % self.steps == 0:
            # Save model with the current timestep in the filename
            if self.verbose > 0:
                print(f"Saving model at step {self.n_calls} to {self.save_path}")
            self.model.save(self.save_path)
        return True

callbacks = CallbackList([
    #TensorboardCallback(),
    #HParamCallback(),
    #SaveOnBestTrainingRewardCallback(1e4,log_dir,path)
    SaveOnStep(1e4, path, verbose=1)
])



### Debug

In [8]:
print(env)

## Agents

### Create agent

In [7]:
model = PPO( # Configure
    policy, 
    env, 
    verbose=verbose,
    device=device,
    tensorboard_log=tensorboard_log)
# DQN
# A2C
# DDPG
# PPO

model.save(path)

/home/dorvan/Documents/dorvan/Babydoll/Produits/Babybot-01/Informatic-01/LearnPyBullet/pybulletEnv/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:284: UserWarning: Path 'ppo_lunar_v1/model' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


### Load agent

In [ ]:

model = A2C.load(path,env,tensorboard_log=tensorboard_log,device=device)
# DQN
# A2C
# DDPG

### Train agent

In [19]:
total_timesteps = 1e5
model = PPO.load(path,env,tensorboard_log=tensorboard_log,device=device)
model.learn(
    total_timesteps, 
    progress_bar=True, 
    reset_num_timesteps=False, 
    tb_log_name=tb_log_name,
    callback=callbacks
    )

model.save(path)

Output()

Saving model at step 60000 to ./ppo_lunar_v1/model/ppo_lunar_v1

Saving model at step 70000 to ./ppo_lunar_v1/model/ppo_lunar_v1

### Monitor 

In [ ]:
!tensorboard --logdir ./logs/tesorboard_log_{name}

### Evaluate agent

In [20]:
eval_env = VecNormalize.load(env_path, DummyVecEnv([make_env]))
eval_env.training = False
eval_env.gamma = 0.0
eval_env.norm_reward = True

n_eval_episodes = 10

mean_reward, std_reward = evaluate_policy(model,eval_env,n_eval_episodes)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=191.42 +/- 143.91551094036555


### Display agent

In [9]:
def make_display_env():
    e = gym.make(env_id, max_episode_steps=max_episode_steps, render_mode='human')
    e = Monitor(e)
    return e

display_env = VecNormalize.load(env_path, DummyVecEnv([make_display_env]))
#display_env = gym.make(env_id, max_episode_steps=max_episode_steps,render_mode='human') #HumanRendering(env)

In [21]:


for ep in range(3):

    # obs, info = display_env.reset() # Unwrapped Env
    obs = display_env.reset()
    while True:
        action, _states = model.predict(obs)

        #obs, reward, terminated, truncated, info = display_env.step(action) # Unwrapped Env
        obs, reward, terminated, info = display_env.step(action)
        terminated = terminated[0]
        truncated = info[0].get('TimeLimit.truncated', False)

        if terminated or truncated:
            break

    time.sleep(0.5)

